# Lab Exercise
## De-Duping Data

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraincentralindia.blob.core.windows.net/

Created user-specific database

Using the database anbasa_cloud_outlook_com_db .

All done!

In [0]:
%run "./Includes/Initialize-Labs"

Initializing lab environment: 
 Declared clearYourResults( passedOnly:Boolean=true ) 
 Declared validateYourSchema( what:String, df:DataFrame, expColumnName:String, expColumnType:String ) 
 Declared validateYourAnswer( what:String, expectedHash:Int, answer:Any ) 
 Declared summarizeYourResults() 
 Declared logYourTest( path:String, name:String, value:Double ) 
 Declared loadYourTestResults( path:String ) returns DataFrame 
 Declared loadYourTestMap( path:String ) returns Map[String,Double]

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

In [0]:
# Answer

# packages 

from pyspark.sql.functions import * 
from pyspark.sql import* 

(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

sourceFile = source + "/dataframes/people-with-dups.txt"
destFile = userhome + "/people.parquet"

# In case it already exists
dbutils.fs.rm(destFile, True)

print(dbutils().fs.head(sourceFile))







Out[6]: False

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [0]:
df = (spark.read
     .option("header", "true")
      .option("inferschema", "true")
      .option("sep", ":")
      .csv(sourceFile)
     )

In [0]:
from pyspark.sql.functions import *

dedupedDf = (df.select
               (col("*"),
                lower(col("firstName")).alias("lcfirstname"),
                lower(col("lastName")).alias("lclastname"),
                lower(col("middleName")).alias("lcmiddlename"),
                translate(col("ssn"),"-", "").alias("ssnNums"))
          .dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "ssnNums", "gender", "birthDate", "salary"])
.drop("lcFirstName", "lcMiddleName", "lcLastName", "ssnNums")
)

##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
display(dedupedDf)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Aaron,Andrea,Mondloch,M,1937-12-02T00:00:00.000+0000,75637,906-59-7221
Aaron,Jermaine,Resler,M,1982-08-26T00:00:00.000+0000,80253,911-19-1232
Aaron,Brady,Morgans,M,1935-10-25T00:00:00.000+0000,283121,912-45-3172
Aaron,Russ,Kopera,M,2008-05-08T00:00:00.000+0000,272069,914-32-4016
Aaron,Willard,Kolden,M,1978-02-07T00:00:00.000+0000,73609,916-12-3224
Aaron,Lesley,Strnad,M,1977-10-25T00:00:00.000+0000,179723,916-43-4368
Aaron,Jonah,Crnich,M,1980-12-09T00:00:00.000+0000,256775,916-48-8115
Aaron,Noe,Arujo,M,1983-05-18T00:00:00.000+0000,163936,918-18-4042
Aaron,Wyatt,Cubito,M,1949-12-10T00:00:00.000+0000,181455,921-39-4145
Aaron,Abraham,Tatters,M,1931-10-06T00:00:00.000+0000,80020,922-54-5718


In [0]:
(dedupedDf.write
.mode("overwrite")
.option("compression", "snappy")
.parquet(destFile)
)

dedupedDf = spark.read.parquet(destFile)
print("Total Records: {0:,} ".format( dedupedDf.count() ))

Total Records: 100,000 


In [0]:
finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()


01 Expected 100000 Records was correct, your answer: 100000


01 Expected 100000 Records:,passed,100000
